In [11]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal
from scipy import stats
from scipy import linalg

In [12]:
class Util:
    def __init(self):
        pass

    @classmethod
    def round_power2(self, x):
        assert x > 0.
        assert not np.iscomplex(x)
        return int(np.power(2., np.ceil(np.log2(x))))

    @classmethod
    def sample_time(self, f_sample, n):
        assert n > 0, 'n must be greater than 0'
        assert f_sample > 0.
        return n / f_sample

    @classmethod
    def dB_power(self, x):
        assert x > 0
        assert not np.iscomplex(x)
        return 10. * np.log10(x)

    @classmethod
    def dB(self, x):
        assert x > 0
        assert not np.iscomplex(x)
        return 20. * np.log10(x)

    @classmethod
    def dB_to_factor_power(self, x):
        assert not np.iscomplex(x)
        return 10.**(x / 10.)

    @classmethod
    def dB_to_factor(self, x):
        assert not np.iscomplex(x)
        return 10.**(x / 20.)

    @classmethod
    def signal_power(self, x, dB=True):
        assert x.ndim == 1
        p = np.sum(np.abs(x)**2.) / len(x)
        if dB:
            return self.dB_power(p)
        else:
            return p

    @classmethod
    def signal_energy(self, x, length):
        assert x.ndim == 1, 'signal must have 1 dimension'
        assert length > 0, 'time length must be greater than 0'
        return self.signal_power(x) * length

    @classmethod
    def snr(self, x, y, dB=True):
        assert x.ndim == 1
        assert y.ndim == 1
        snr = self.signal_power(x, dB=False) / self.signal_power(y, dB=False)
        if dB:
            return self.dB_power(snr)
        else:
            return snr

    @classmethod
    def get_signal_length(self, fs, seconds):
        assert fs > 0. and seconds > 0.
        return fs * seconds

In [13]:
class WirelessMicrophone:
    def __init__(self, f_sample=1000.0, num_samples=None, t_sec=None):
        assert f_sample > 0, 'must be greater than 0'
        if num_samples is not None:
            assert num_samples > 0, 'must be greater than 0'
            self.f_sample = f_sample
            self.num_samples = int(num_samples)
            self.t_sec = self.num_samples / self.f_sample
        elif t_sec is not None:
            assert t_sec > 0, 'must be greater than 0'
            self.f_sample = f_sample
            self.t_sec = t_sec
            self.num_samples = int(self.t_sec * self.f_sample)
        else:
            assert False, 'either num_samples or t_sec needed'

    def get_signal(self, f_center, f_deviation, f_modulation, dB=0.):
        t = np.arange(self.num_samples) / self.f_sample
        x = np.exp(1.j *
                   (2. * np.pi * f_center * t + f_deviation / f_modulation *
                    np.sin(2. * np.pi * f_modulation * t)))
        x -= np.mean(x)  # normalize signal
        x /= np.std(x)  # ^
        x *= 10.**(dB / 20.)
        return x

    def get_silent(self, f_center, dB=0.):
        return self.get_signal(f_center, 5000, 32000, dB)

    def get_soft(self, f_center, dB=0.):
        return self.get_signal(f_center, 3900, 15000, dB)

    def get_loud(self, f_center, dB=0.):
        return self.get_signal(f_center, 13400, 32600, dB)

In [14]:
class WhiteGaussianNoise:
    def __init__(self, f_sample=1000.0, n=None, t_sec=None):
        assert f_sample > 0, 'must be greater than 0'
        if n is not None:
            assert n > 0, 'must be greater than 0'
            self.f_sample = f_sample
            self.n = int(n)
            self.t_sec = self.n / self.f_sample
        elif t_sec is not None:
            assert t_sec > 0, 'must be greater than 0'
            self.f_sample = f_sample
            self.t_sec = t_sec
            self.n = int(self.t_sec * self.f_sample)
        else:
            assert False, 'either n or t_sec needed'

    def get_signal(self, dB=0.):
        x = 10.**(dB / 10.)
        x = stats.multivariate_normal(mean=[0., 0.],
                                      cov=[[.5 * x, 0.], [0., .5 * x]])
        x = x.rvs(size=self.n).view(np.complex128).reshape(self.n)
        return x

In [22]:
%%script false --no-raise-error

# Example
sample_frequency = 1e6  # in Hz
length = 1.0  # in s
signal_strength = 0.0  # in dB
noise_strength = 0.0  # in dB

wm = WirelessMicrophone(f_sample=sample_frequency, t_sec=length)
# wm = WirelessMicrophone(f_sample=sample_frequency, num_samples=1e5)
sig = wm.get_signal(f_center=1e5,
                    f_deviation=15000,
                    f_modulation=3900,
                    dB=signal_strength)

wgn = WhiteGaussianNoise(f_sample=sample_frequency, t_sec=length)
noise = wgn.get_signal(dB=noise_strength)

both = sig + noise

print('wm.f_sample     %6.2f' % (wm.f_sample))
print('wm.nnum_samples %6.2f' % (wm.num_samples))
print('wm.t_sec        %6.2f' % (wm.t_sec))

num_samples = wm.num_samples
sample_freq = wm.f_sample

print('Signal power:   %6.2f dB' % (Util.signal_power(sig, dB=True)))
print('Noise power:    %6.2f dB' % (Util.signal_power(noise, dB=True)))
print('SNR:            %6.2f dB' % (Util.snr(sig, noise, dB=True)))
print('Mean:           %6.2f ' % (np.abs(noise.mean())))
print('Var:            %6.2f ' % (np.abs(noise.var())))